<!-- サンチャゴの大暴落で勝つから引用 -->
<!-- 移動平均線からの乖離が大きい銘柄に対してエントリー -->

In [1]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("../")
from common.evaluate.evaluate import evaluation_index
import codecs
from joblib import Parallel, delayed
import pandas_datareader.data as web
import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import mplfinance as mpf


In [2]:

# # 日経平均株価を取得する
# nikkei = web.DataReader("NIKKEI225", "fred", "2017/01/04")
# for sma in [5,25,45,75]:
#     nikkei[f"sma{sma}"] = nikkei['NIKKEI225'].rolling(sma).mean()
#     nikkei[f'sma_close_ratio_{sma}'] = (nikkei['NIKKEI225'] / nikkei[f"sma{sma}"] - 1) * 100
# display(nikkei)

In [3]:
topix_code = pd.read_csv('../data/stock_code/TOPIX_weight_jp.csv',index_col=0)
topix_code = topix_code.sort_values('TOPIXに占める個別銘柄のウェイト',ascending=False)
display(topix_code)

topix_code = topix_code['銘柄名'].values.tolist()

number_of_stocks = 500
print(topix_code[:number_of_stocks])

,銘柄名,コード,業種,TOPIXに占める個別銘柄のウェイト,ニューインデックス区分,調整係数対象銘柄
日付,,,,,,
20220331,トヨタ自,7203.0,輸送用機器,4.23%,TOPIX Core30,NaN
20220331,ソニーグループ,6758.0,電気機器,3.00%,TOPIX Core30,NaN
20220331,キーエンス,6861.0,電気機器,1.95%,TOPIX Core30,NaN
20220331,三菱ＵＦＪ,8306.0,銀行業,1.77%,TOPIX Core30,NaN
20220331,東エレク,8035.0,電気機器,1.51%,TOPIX Core30,NaN
...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN


['トヨタ自', 'ソニーグループ', 'キーエンス', '三菱ＵＦＪ', '東エレク', 'リクルートＨＤ', 'ＮＴＴ', '任天堂', '三菱商事', 'ソフトバンクグループ', '信越化', '日立', 'ＫＤＤＩ', '伊藤忠', 'ダイキン工', 'ホンダ', '武田薬', 'ＨＯＹＡ', 'ＯＬＣ', '三井住友', '三井物産', '日本電産', '村田製', '東京海上', '７＆ｉＨＤ', 'ソフトバンク', '第一三共', 'ＳＭＣ', 'みずほ', 'ファナック', 'デンソー', 'アステラス薬', '富士通', 'キヤノン', 'オリックス', '三菱電', 'ブリヂス', '丸紅', '住友商', '富士フイルム', 'コマツ', 'ＪＲ東海', '中外薬', 'パナソニック', 'ＪＲ東', '第一生命ＨＤ', 'オリンパス', '三井不', '三菱地所', '資生堂', 'ＪＴ', 'シマノ', '花王', '日本郵政', 'クボタ', 'テルモ', 'アサヒ', 'イオン', '京セラ', 'スズキ', '塩野義薬', '日本製鉄', '大和ハウス', 'レーザーテック', 'ＳＯＭＰＯＨＤ', '大塚ＨＤ', 'ＭＳ＆ＡＤ', '郵船', 'ユニチャーム', 'バンダイナムコ', 'ファーストリテ', '東芝', 'セコム', '積水ハウス', 'Ｚホールディングス', 'ＮＲＩ', 'ルネサス', '野村', '味の素', '住友鉱', 'エムスリー', '三住トラスト', 'シスメックス', 'ＩＮＰＥＸ', 'アドバンテス', 'オムロン', 'ＥＮＥＯＳ', '住友不', '小野薬', 'エーザイ', '豊田織', '旭化成', 'キリンＨＤ', 'ＮＥＣ', 'ＮＴＴデータ', 'ＴＤＫ', '三菱重', 'ネクソン', '日産自', 'ＪＲ西', 'ニトリＨＤ', 'ＡＮＡ', 'キッコマン', 'りそなＨＤ', 'ＪＰＸ', '島津製', 'オービック', 'ＳＵＢＡＲＵ', '日東電', '住友電工', '商船三井', '三菱ケミＨＤ', 'いすゞ自', 'ＪＡＬ', '豊田通商', 'ディスコ', '大和証Ｇ', '電通グループ', '安川電', 'ダイフク', '浜松ホトニク', 'ヤクルト', 'マキタ',

In [4]:
candle_data_path = "../data/candle_data/TOPIX"
os.makedirs(candle_data_path, exist_ok=True)

In [5]:

# files = os.listdir('../data/raw_data/candle_data/TOPIX/')
# print(len(files))
# for file in topix_code[:number_of_stocks]:
#     # try:
#     data = pd.read_csv('../data/raw_data/candle_data/TOPIX/'+file + '.csv',index_col=0)
#     # print(data.iloc[0])
#     print(file)
#     data = data.rename(columns={'Open': 'open', 'High': 'high','Low': 'low','Close': 'cl','Adj Close':'close','Volume': 'volume'})
#     # display(data)
#     data['code'] = file

#     data['price'] = data['close'].shift(-1) 
#     for sma in [5, 25]:
#         data[f"sma{sma}"] = data['close'].rolling(sma).mean()
#         data[f"sma_tilt{sma}"] = data[f"sma{sma}"] / data[f"sma{sma}"].shift(1) - 1
#         data[f'sma_close_ratio_{sma}'] = (data['close'] / data[f"sma{sma}"] - 1) * 100
#     data.to_csv(candle_data_path+'/'+file + '.csv')
#     # except:
#     #     continue


In [6]:
code_name = 'トヨタ自'
df = pd.read_csv(candle_data_path+'/'+code_name+'.csv',index_col = 0)
df = df['2020-01-04':]
display(df)

,high,low,open,cl,volume,close,code,price,sma5,sma_tilt5,sma_close_ratio_5,sma25,sma_tilt25,sma_close_ratio_25
Date,,,,,,,,,,,,,,
2020-01-06,1525.199951,1506.000000,1519.800049,1513.000000,33361500.0,1269.066284,トヨタ自,1294.229492,1288.794189,-0.004561,-1.530726,1296.041182,-0.000740,-2.081330
2020-01-07,1544.400024,1523.800049,1530.800049,1543.000000,24803500.0,1294.229492,トヨタ自,1277.789429,1290.102686,0.001015,0.319882,1296.208940,0.000129,-0.152711
2020-01-08,1528.400024,1508.199951,1513.000000,1523.400024,28061000.0,1277.789429,トヨタ自,1286.680542,1286.479175,-0.002809,-0.675467,1295.490947,-0.000554,-1.366395
2020-01-09,1538.000000,1527.599976,1534.400024,1534.000000,21265000.0,1286.680542,トヨタ自,1285.170654,1284.365479,-0.001643,0.180250,1295.705679,0.000166,-0.696542
2020-01-10,1539.599976,1530.199951,1539.000000,1532.199951,17417000.0,1285.170654,トヨタ自,1298.255615,1282.587280,-0.001384,0.201419,1295.188994,-0.000399,-0.773504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-11,2210.000000,2050.500000,2205.500000,2082.000000,64821200.0,2082.000000,トヨタ自,2050.000000,2201.900000,-0.013751,-5.445297,2180.020000,-0.002571,-4.496289
2022-05-12,2114.500000,2011.500000,2050.000000,2050.000000,44104700.0,2050.000000,トヨタ自,2055.000000,2166.300000,-0.016168,-5.368601,2173.820000,-0.002844,-5.695964
2022-05-13,2086.500000,2028.500000,2043.000000,2055.000000,31138400.0,2055.000000,トヨタ自,2049.500000,2122.100000,-0.020403,-3.161962,2167.580000,-0.002871,-5.193811


In [7]:
def calcurate_prediction(date, sma, entry_threshold):   
    sma_close_ratio_list = []
    price_list = []
    code_list = []
    result = pd.DataFrame()
    for file in topix_code[:number_of_stocks]:
        data = pd.read_csv(candle_data_path+'/'+file + '.csv',index_col=0)
        try:
            data = data.loc[[date]]
            result = pd.concat([result,data],axis=0)
        except:
            continue

    # トレンドが上向きのみ抽出
    result = result[result[f"sma_tilt{sma}"] > 0]
    result = result.sort_values(f'sma_close_ratio_{sma}',ascending=True)

    # 騰落率条件を満たす銘柄の数
    # result_ = result[result[f'sma_close_ratio_{sma}'] >= -entry_threshold]
    # display(result_)

    if len(result) > 0:
        return result.iloc[0]
    else:
        return None


In [8]:
def backtest(df,exit_range,sma,entry_threshold):
    entry_history = []
    code = ''
    df = df.rename(columns={'Open': 'open', 'High': 'high','Low': 'low','Close': 'close','Volume': 'volume'})
    n = df['close'].values.size
    y = df['close'].values.copy() * 0.0
    poss = df['close'].values.copy() * 0.0
    ret = 0.0
    pos = 0.0
    buy_entry_price = 1.0
    sell_entry_price = 1.0
    buy_entry_prices = np.full((n), np.nan)
    sell_entry_prices = np.full((n), np.nan)
    buy_exit_prices = np.full((n), np.nan)
    sell_exit_prices = np.full((n), np.nan)
    for i in tqdm.tqdm(range(len(df))):
        prev_pos = pos
        date = df.index[i]

        # Exit of long
        if prev_pos == 1 and (i - entry_date) >= exit_range:
            data = pd.read_csv(candle_data_path+'/'+code + '.csv',index_col=0)
            data = data.rename(columns={'Open': 'open', 'High': 'high','Low': 'low','Close': 'close','Volume': 'volume'})

            data['price'] = data['close'].shift(-1) 
            sell_exit_price = data.loc[date,'price']
            sell_exit_prices[i] = data.loc[date,'price']
            ret += (sell_exit_price / buy_entry_price - 1)
            pos -= 1
            print(f'EXIT :date:{date}, code:{code}, sell_exit_price:{sell_exit_price}, return:{100 * round(sell_exit_price / buy_entry_price - 1,2)} %')


        # entry
        if prev_pos == 0:
            result = calcurate_prediction(date, sma, entry_threshold)
            if not result is None:
                sma_close_ratio = result[f'sma_close_ratio_{sma}']

                if sma_close_ratio <= entry_threshold:
                    buy_entry_price = result['price']
                    buy_entry_prices[i] = result['price']
                    code = result['code']
                    entry_date = i
                    pos += 1
                    print(f'ENTRY : date:{date}, code:{code}, buy_entry_price:{buy_entry_price}, sma_close_ratio:{sma_close_ratio}')
                    entry_history.append([date, code, buy_entry_price])


        y[i] = ret
        poss[i] = pos

    return y, poss, buy_entry_prices, sell_entry_prices, buy_exit_prices, sell_exit_prices, entry_history


In [9]:
def plot_entry_history(df):
    for i in range(len(df)):
        date = df.iloc[i,df.columns.get_loc('date')]
        code = df.iloc[i,df.columns.get_loc('code')]
        buy_entry_price = df.iloc[i,df.columns.get_loc('buy_entry_price')]
        print(code)
        print(type(code))

        data = pd.read_csv(candle_data_path+'/'+code + '.csv',index_col=0)
        data = data.rename(columns={'Open': 'open', 'High': 'high','Low': 'low','Close': 'close','Volume': 'volume'})

        data['predict_buy_entry'] = np.nan
        data.loc[date,'predict_buy_entry'] = buy_entry_price
        data = data.iloc[data.index.get_loc(date)-50:data.index.get_loc(date)+50]
        data.index = pd.to_datetime(data.index)
        

        addplot_2s = mpf.make_addplot(data['predict_buy_entry'], type="scatter", color="red")
        mckt_colors = mpf.make_marketcolors(up='red',down='blue',inherit=True)
        styl  = mpf.make_mpf_style(base_mpl_style='seaborn',marketcolors=mckt_colors)
        mpf.plot(data, type="candle", style=styl, mav=(5,21), addplot=addplot_2s)

    

In [12]:

# # 単一条件
# exit_range = 5
# sma = 5
# entry_threshold = 0
# # try:
# print(f'exit_range:{exit_range}, sma:{sma}, entry_threshold:{entry_threshold}')
# cumulative_return, possition, buy_entry_prices, sell_entry_prices, buy_exit_prices, sell_exit_prices, entry_history = backtest(df,exit_range,sma,entry_threshold)
# df_result =  pd.DataFrame(
#     {
#         "cumulative_return": cumulative_return,
#         "position": possition,
#         "buy_entry_price": buy_entry_prices,
#         "sell_entry_price": sell_entry_prices,
#         "buy_exit_price": buy_exit_prices,
#         "sell_exit_price": sell_exit_prices,
#     },
#     index=df.index
# )
# entry_history_df = pd.DataFrame(entry_history,
#                   columns=['date', 'code', 'buy_entry_price'])

# result_dict = evaluation_index(df_result)
# plot_entry_history(entry_history_df)

for exit_range in [5]:
    for sma in [25]:
        for entry_threshold in [-5,-15,-20,-35]:
            try:
                print('==================')
                print(f'exit_range:{exit_range}, sma:{sma}, entry_threshold:{entry_threshold}')

                cumulative_return, possition, buy_entry_prices, sell_entry_prices, buy_exit_prices, sell_exit_prices, entry_history = backtest(df,exit_range,sma,entry_threshold)
                df_result =  pd.DataFrame(
                    {
                        "cumulative_return": cumulative_return,
                        "position": possition,
                        "buy_entry_price": buy_entry_prices,
                        "sell_entry_price": sell_entry_prices,
                        "buy_exit_price": buy_exit_prices,
                        "sell_exit_price": sell_exit_prices,
                    },
                    index=df.index
                )
                entry_history_df = pd.DataFrame(entry_history,
                  columns=['date', 'code', 'buy_entry_price'])

                result_dict = evaluation_index(df_result)
                # plot_entry_history(entry_history_df)

            except:
                print('error')
                continue


df_result['cumulative_return'].plot()
plt.title('cumulative_return')
plt.show()

print('ポジション推移です。変動が細かすぎて青色一色になっていると思います。')
print('ちゃんと全ての期間でトレードが発生しているので、正常です。')
df_result['position'].plot()
plt.title('position')
plt.show()

print('ポジションの平均の推移です。どちらかに偏りすぎていないかなどを確認できます。')
df_result['position'].rolling(1000).mean().plot()
plt.title('position_average')
plt.show()

print('取引量(ポジション差分の絶対値)の累積です。')
print('期間によらず傾きがだいたい同じなので、全ての期間でちゃんとトレードが行われていることがわかります。')
df_result['position'].diff(1).abs().dropna().cumsum().plot()
plt.title('cumulative_position')

exit_range:5, sma:25, entry_threshold:-5


  2%|▏         | 10/575 [00:24<22:15,  2.36s/it]

ENTRY : date:2020-01-20, code:マネーフォワード, buy_entry_price:2420.0, sma_close_ratio:-6.114922472934913
EXIT :date:2020-01-27, code:マネーフォワード, sell_exit_price:2415.0, return:-0.0 %


  3%|▎         | 19/575 [00:33<14:26,  1.56s/it]

ENTRY : date:2020-01-31, code:Ｆ＆ＬＣ, buy_entry_price:2324.682861328125, sma_close_ratio:-5.099967515545433
EXIT :date:2020-02-07, code:Ｆ＆ＬＣ, sell_exit_price:2356.866943359375, return:1.0 %


...

 27%|██▋       | 154/575 [02:43<07:18,  1.04s/it]

ENTRY : date:2020-08-21, code:ベイカレント, buy_entry_price:11455.9052734375, sma_close_ratio:-5.643293478848156
EXIT :date:2020-08-28, code:ベイカレント, sell_exit_price:11242.982421875, return:-2.0 %


 29%|██▊       | 165/575 [02:59<12:41,  1.86s/it]

ENTRY : date:2020-09-07, code:日電子, buy_entry_price:2986.925537109375, sma_close_ratio:-6.706021457523438
EXIT :date:2020-09-14, code:日電子, sell_exit_price:3210.20068359375, return:7.000000000000001 %


 30%|███       | 173/575 [03:07<09:05,  1.36s/it]